# pykabu-calendar Quick Start

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pykabu-calendar/blob/main/examples/quickstart.ipynb)

This notebook demonstrates how to use pykabu-calendar to get Japanese stock earnings calendar data.

## Installation

Install pykabu-calendar from PyPI:

In [ ]:
!pip install -q pykabu-calendar

## Setup

Find a future date with earnings announcements:

In [ ]:
import pykabu_calendar as cal
from pykabu_calendar.earnings.sources import MatsuiEarningsSource
from datetime import datetime, timedelta

_matsui = MatsuiEarningsSource()

def find_date_with_earnings(max_days=30):
    """Find a future date that has earnings announcements."""
    today = datetime.now()
    for i in range(1, max_days + 1):
        target = today + timedelta(days=i)
        date_str = target.strftime("%Y-%m-%d")
        try:
            # Use only matsui (tradersweb may block cloud IPs)
            df = _matsui.fetch(date_str)
            if len(df) > 10:  # Found a date with meaningful data
                print(f"Found {len(df)} earnings on {date_str}")
                return date_str
        except Exception:
            continue
    # Fallback: use 14 days from now
    return (today + timedelta(days=14)).strftime("%Y-%m-%d")

# Find a good target date
TARGET_DATE = find_date_with_earnings()
print(f"Using target date: {TARGET_DATE}")

## Basic Usage

Get the earnings calendar for the target date:

> **Note**: Some sources (like Tradersweb) may block requests from cloud environments like Colab. The library handles this gracefully and continues with available sources.

In [ ]:
# Get calendar (all sources are lightweight - no browser needed)
# If running in Colab, tradersweb may return 403 - that's OK, other sources are used
df = cal.get_calendar(TARGET_DATE, sources=["matsui", "tradersweb"], infer_from_history=False)

print(f"Found {len(df)} earnings announcements for {TARGET_DATE}")
df.head(10)

## Output Schema

The calendar returns these columns:

| Column | Description |
|--------|-------------|
| `code` | Stock code (e.g., "7203") |
| `name` | Company name |
| `datetime` | Best estimate announcement datetime |
| `candidate_datetimes` | List of candidate datetimes (most likely first) |
| `sbi_datetime` | Datetime from SBI (if available) |
| `matsui_datetime` | Datetime from Matsui |
| `tradersweb_datetime` | Datetime from Tradersweb |
| `inferred_datetime` | Datetime inferred from historical patterns |
| `past_datetimes` | List of past earnings announcement datetimes |

In [ ]:
# View key columns
df[["code", "name", "datetime", "matsui_datetime"]].head(10)

## Historical Inference

By default, pykabu-calendar infers announcement times based on past patterns using [pykabutan](https://pypi.org/project/pykabutan/).

You can disable this for faster results:

In [ ]:
# With inference enabled (default), check inferred times and past patterns
# This takes longer as it fetches historical data for each company
df_inferred = cal.get_calendar(TARGET_DATE, sources=["matsui"], infer_from_history=True)

# View companies with inferred datetimes
print(f"Found {len(df_inferred)} companies with inference")
df_inferred[["code", "name", "datetime", "inferred_datetime"]].head(10)

## Data Sources

Different sources have different strengths:

| Source | Notes |
|--------|-------|
| `sbi` | JSONP API, most comprehensive, fast |
| `matsui` | Lightweight, fast, works everywhere |
| `tradersweb` | May block cloud IPs (Colab, etc.) |

In [ ]:
# Using only Matsui (recommended for Colab)
df_matsui = cal.get_calendar(
    TARGET_DATE,
    sources=["matsui"],
    infer_from_history=False
)
print(f"Matsui only: {len(df_matsui)} companies")

In [ ]:
# By default, all sources are used (sbi, matsui, tradersweb).
# All sources use lightweight HTTP requests - no browser needed.
# To run with specific sources only:
# df = cal.get_calendar(TARGET_DATE, sources=["matsui", "tradersweb"])

## Individual Sources

You can also use individual source classes directly:

In [ ]:
# Get data from Matsui directly
df_matsui = _matsui.fetch(TARGET_DATE)
print(f"Matsui: {len(df_matsui)} companies")
df_matsui.head()

## Inference Utilities

Use inference functions to predict announcement times based on historical patterns:

In [ ]:
# Get past earnings announcement times for a stock
past = cal.get_past_earnings("7203")  # Toyota
print(f"Past earnings announcements for Toyota (7203):")
for dt in past[:5]:
    print(f"  {dt}")

In [ ]:
# Infer datetime for a future announcement
inferred_dt, confidence, past_dts = cal.infer_datetime("7203", TARGET_DATE)
print(f"Inferred datetime: {inferred_dt}")
print(f"Confidence: {confidence}")

## Export to CSV

Export the calendar to CSV for use in Excel or Google Sheets:

In [ ]:
# Export to CSV with proper encoding for Japanese characters
output_file = f"earnings_{TARGET_DATE}.csv"
cal.export_to_csv(df, output_file)

print(f"Exported to {output_file}")

In [ ]:
# Show all available columns
print("Available columns:")
print(df.columns.tolist())